<a href="https://colab.research.google.com/github/hiu04/NLP-Deep-Learning/blob/main/Copy_of_mmai5400_assign_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install -U pip
# !pip install -U sentencepiece
# !pip install -U transformers

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
file_path = '/content/A3_reviews.csv'
data = pd.read_csv('A3_reviews.csv', sep = '\t')

data.head()

In [ ]:
import zipfile
import os  # Import the os module

zipfile_path = '/content/model-best.zip'  # Update the path if necessary
extracted_folder_path = '/content'

with zipfile.ZipFile(zipfile_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

# Checking the contents of the extracted folder to identify the model structure
extracted_files = os.listdir(extracted_folder_path)
extracted_files

['.config', 'model-best', 'model-best.zip', 'A3_reviews.csv', 'sample_data']

## NER model

Use the NER model to locate the dishes in a given review.

In [ ]:
import spacy

In [ ]:
from pathlib import Path
from spacy.util import load_model_from_path

# Convert string path to Path object
model_path = Path("/content/model-best")

# Load the custom NER model
ner_model = load_model_from_path(model_path)

/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.5.1 and may not be 100% compatible with the current version (3.6.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [ ]:
def extract_entities(text, model):
    doc = model(text)
    return [(ent.text, ent.label_) for ent in doc.ents]


In [ ]:
# Apply the function to the 'Review' column
data['extracted_entities'] = data['Review'].apply(lambda x: extract_entities(x, ner_model))

## Add tags to the review

Add tags to the review that indicates the start and end of the dish. Example:
"Best [B-ASP]tandoori chicken[E-ASP] I've ever had."

In [ ]:
def tag_reviews(df, ner_model):
    tagged_reviews = []
    for review in df['Review']:
        doc = ner_model(review)
        tagged_review = review
        offset = 0
        for ent in doc.ents:
            start, end = ent.start_char + offset, ent.end_char + offset
            tagged_review = tagged_review[:start] + '[B-ASP]' + ent.text + \
            '[E-ASP]' + tagged_review[end:]
            offset += len('[B-ASP][E-ASP]')
        tagged_reviews.append(tagged_review)
    return tagged_reviews

data['Tagged_Review'] = tag_reviews(data, ner_model)

## Pass Reviews to absa

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, \
pipeline

# Load the ABSA model and tokenizer
model_name = "yangheng/deberta-v3-base-absa-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast = False)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
classifier = pipeline("text-classification", model = model,
                      tokenizer = tokenizer)

tokenizer_config.json:   0%|          | 0.00/372 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/738M [00:00<?, ?B/s]

### Define absa() model

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, \
pipeline
from pathlib import Path

model_path = Path("/content/model-best")

# Define the ABSA function
def absa(reviews):
    # Load the custom NER model
    ner_model = spacy.load(model_path)

    # ABSA model setup
    absa_model = "yangheng/deberta-v3-base-absa-v1.1"
    tokenizer = AutoTokenizer.from_pretrained(absa_model, use_fast = False)
    model = AutoModelForSequenceClassification.from_pretrained(absa_model)
    classifier = pipeline("text-classification", model = model,
                          tokenizer = tokenizer)

    def tag_dishes(review):
        predicted_dish_entities = ner_model(review)
        dishes = [
            ent.text for ent in predicted_dish_entities.ents if \
            ent.label_ == 'DISH'
        ]
        return dishes

    def analyze_sentiment(review_text, dish):
        analysis = classifier(review_text, text_pair = dish)
        return {
            'sentiment': analysis[0]['label'],
            'confidence': analysis[0]['score']
        }

    results = []
    for idx, review in enumerate(reviews):
        review_text = review
        dishes = tag_dishes(review_text)
        for dish in dishes:
            sentiment_info = analyze_sentiment(review_text, dish)
            results.append({
                'review_id': idx,
                'dish': dish,
                'sentiment': sentiment_info['sentiment'],
                'confidence': sentiment_info['confidence']
            })

    results_df = pd.DataFrame(results)
    return results_df


In [ ]:
# Extract the first review texts into a list
first_reviews = data['Review'].head(1).tolist()

# Apply the absa function to this list of reviews
results_df = absa(first_reviews)

# Display the resulting DataFrame
print(results_df)


/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.5.1 and may not be 100% compatible with the current version (3.6.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


   review_id             dish sentiment  confidence
0          0             guac  Positive    0.979973
1          0  tacos al pastor  Positive    0.888602
2          0            tacos  Positive    0.890640
3          0   tortilla chips  Positive    0.974422
4          0            tacos  Positive    0.890640
5          0            tacos  Positive    0.890640
